# 🧬 CpG Island Finder & GC/CpG Ratio Plotter

This notebook reads a **DNA FASTA sequence**, scans it in sliding windows, computes **GC content** and **CpG observed/expected ratio**, and identifies regions that qualify as **CpG islands**.

It also generates:
- 📊 GC content plot
- 📊 CpG ratio plot
- 📝 Table of detected CpG islands

---

## Criteria for CpG islands
- GC content > **50%**
- CpG observed/expected ratio > **0.6**
- Length > **200 bp**


In [ ]:
!pip install biopython pandas matplotlib seaborn

In [ ]:
from Bio import SeqIO
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

## 📂 Upload FASTA file
Upload a DNA FASTA file (single sequence) to scan for CpG islands.

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(f"Loaded: {filename}")

In [ ]:
record = next(SeqIO.parse(filename, "fasta"))
sequence = str(record.seq).upper()
print(f"Sequence length: {len(sequence)} bp")

## 🔍 Compute GC content & CpG ratio in sliding windows
- Using window size = **200 bp**
- Step size = **50 bp**

In [ ]:
window_size = 200
step_size = 50

gc_contents = []
cpg_ratios = []
positions = []

for start in range(0, len(sequence) - window_size + 1, step_size):
    window_seq = sequence[start:start+window_size]
    gc = (window_seq.count('G') + window_seq.count('C')) / len(window_seq)

    c_count = window_seq.count('C')
    g_count = window_seq.count('G')
    cg_count = sum(1 for i in range(len(window_seq)-1) if window_seq[i:i+2] == "CG")

    exp_cpg = (c_count * g_count) / len(window_seq) if len(window_seq) else 0
    cpg_ratio = (cg_count / exp_cpg) if exp_cpg else 0

    gc_contents.append(gc)
    cpg_ratios.append(cpg_ratio)
    positions.append(start)

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(12,5))
plt.plot(positions, [gc*100 for gc in gc_contents], label="GC Content (%)")
plt.axhline(50, color='red', linestyle='--', label="50% GC threshold")
plt.ylabel("GC Content (%)")
plt.xlabel("Position")
plt.legend()
plt.show()

plt.figure(figsize=(12,5))
plt.plot(positions, cpg_ratios, label="CpG Observed/Expected Ratio")
plt.axhline(0.6, color='purple', linestyle='--', label="0.6 CpG threshold")
plt.ylabel("CpG Ratio")
plt.xlabel("Position")
plt.legend()
plt.show()

## 🧬 Identify CpG islands
Regions meeting:
- GC > **50%**
- CpG ratio > **0.6**
- Length >= **200 bp**


In [ ]:
islands = []
for pos, gc, cpg in zip(positions, gc_contents, cpg_ratios):
    if gc > 0.5 and cpg > 0.6:
        islands.append({
            "Start": pos,
            "End": pos + window_size,
            "Length": window_size,
            "GC Content (%)": round(gc*100,2),
            "CpG Ratio": round(cpg,2)
        })

df_islands = pd.DataFrame(islands)
df_islands

In [ ]:
df_islands.to_csv("cpg_islands_results.csv", index=False)
files.download("cpg_islands_results.csv")

# ✅ Summary
- ✅ Computed sliding window GC content & CpG ratio.
- ✅ Identified regions matching standard CpG island criteria.
- ✅ Plotted GC content & CpG ratio across the sequence.
- ✅ Saved results to a CSV file for further analysis.


